In [1]:
import requests # type: ignore
import json
import csv
import pandas as pd

In [2]:
# Load and read json file
with open("request_data.json", "r") as file:
    request_data = json.load(file)

wanted_list = request_data["url"]
request_headers = request_data["headers"]

In [3]:
# Get session
session = requests.Session()
response = session.get(url=wanted_list, headers=request_headers)
data = json.loads(response.text)
session.close()

In [4]:
# Extract base information
name_wanted = [item["title"] for item in data["items"]]
age_max_wanted = [item["age_max"] for item in data["items"]]
age_min_wanted = [item["age_min"] for item in data["items"]]
sex_wanted = [item["sex"] for item in data["items"]]

# Extract category information
occupation_wanted = [item["occupations"] for item in data["items"]]
location_wanted = [item["locations"] for item in data["items"]]
place_of_birth_wanted = [item["place_of_birth"] for item in data["items"]]
subjects_wanted = [item["subjects"] for item in data["items"]]

# Extract other information
details_wanted = [item["details"] for item in data["items"]]

In [5]:
# Insert extract values in csv file
data_dict = {"name": name_wanted, "age_max": age_max_wanted, "age_min": age_min_wanted,
    "sex": sex_wanted, "occupation": occupation_wanted, "location": location_wanted,
    "place_of_birth":place_of_birth_wanted, "subjects": subjects_wanted, "details":details_wanted}

df = pd.DataFrame(data_dict)
df.to_csv("data_fbi.csv", index=False)